# Verificacion de Estimaciones para la Campaña 2023.

In [106]:
import pandas as pd
import numpy as np
import math

In [107]:
df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')

/var/folders/6c/17qbm4nd5318hd4bcrx68wjh0000gn/T/ipykernel_66877/1758670802.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')


In [108]:
df.columns

Index(['Unnamed: 0', 'Ruta', 'Sitio', 'Año', 'Camara', 'Extra', 'Archivo',
       'Fecha', 'Hora', 'Animal_proba', 'Animal', 'Guanaco_proba', 'Guanaco',
       'Especie', 'Cantidad_proba', 'Cantidad', 'Validar', 'Validado'],
      dtype='object')

In [109]:
df.shape

(118765, 18)

## Estimacion de Proporciones:

In [110]:
# Proporcion de Imagenes con Animales
pAnimal = sum(df['Animal']==1)/len(df)
print('Proporcion de Imagenes con Animales: ', round(pAnimal * 100,2), '%')

Proporcion de Imagenes con Animales:  29.2 %


In [111]:
# Proporcion de Imagenes sin Animales
pNoAnimal = sum(df['Animal']==0)/len(df)
print('Proporcion de Imagenes sin Animales: ', round(pNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales:  70.8 %


In [112]:
pGuanaco = (49.19/100)

## Tiempo Total del Proceso:

### Tiempo de Ejecucion Automatica:

La eficiencia en los tiempos se vio perjudica por el uso de AWS para almacenar las imagenes. Al tener mayor latencia el proceso de consultar las imagenes demora más. Ademas, para consultar las imagenes se debieron tensorificar y extraer los metadatos antes de aplicar los modelos.

Se utilizaron multiprocesos concurrentes para hacer el proceso mas rapido. Y tecnicas de autoguardado para que en caso de que se rompa el proceso no se tenga que iniciar desde cero.

In [113]:
tAutom =1/4.18
TAutom = (len(df) * (tAutom))/(60*60)
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos por imagen')
print('Tiempo de Automatizacion: ', math.ceil(TAutom/8), 'jornadas')

Tiempo de Automatizacion:  0.24 segundos por imagen
Tiempo de Automatizacion:  1 jornadas


Por lo que el tiempo total del proceso automatico es de **7.89 horas**  que son equivalentes a **1 jornada**.

### Tiempo de Validacion:

El porcantaje de imagenes a validar sobre el total de la base es de:

In [114]:
pValNoAnimal = sum((df['Validar']==1) & (df['Animal']==0))/sum(df['Animal']==0)
print('Proporcion de Imagenes sin Animales a Validar: ', round(pValNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales a Validar:  13.93 %


In [115]:
pValAnimal = sum((df['Validar']==1) & (df['Animal']==1))/sum(df['Animal']==1)
print('Proporcion de Imagenes con Animales a Validar: ', round(pValAnimal * 100,2), '%')

Proporcion de Imagenes con Animales a Validar:  79.58 %


In [116]:
# Porcentaje de Imagenes a Validar:
pVal = sum(df['Validar']==1)/len(df)
print('Porcentaje de Imagenes a Validar: ', round(pVal * 100,2), '%')

Porcentaje de Imagenes a Validar:  33.1 %


In [117]:
# Tiempo de Validacion:
tVal = pVal * 1.36
TVal = (len(df) * tVal)/(60*60)
print('Tiempo de Validacion: ', round(TVal,2), 'horas')
print('Tiempo de Validacion: ', math.ceil(TVal/8), 'jornadas')

Tiempo de Validacion:  14.85 horas
Tiempo de Validacion:  2 jornadas


Por lo que el tiempo total del proceso de validacion es de **14.85 horas**  que son equivalentes a **2 jornadas**.

### Tiempo Total

El tiempo total del proceso sera de:

In [118]:
tTotal = tAutom + tVal
TTotal = TAutom + TVal
print('Tiempo del Proceso: ', round(TTotal,2), 'horas')
print('Tiempo del Proceso: ', round(TTotal/8,2), 'jornadas')

Tiempo del Proceso:  22.74 horas
Tiempo del Proceso:  2.84 jornadas


El proceso completo requiere **2.84 jornadas laborales**  (23 horas). Cabe destacar que de estas solo **15 horas requieren atencion humana**

### Cantidad de Imagenes por Hora:

Para calcular la cantidad de imágenes que se procesaran por hora se tiene en cuenta tanto el tiempo que tarda en procesarlas los modelos predictivos cómo el tiempo que se tarda en validar aquellas que necesiten un refuerzo en su clasificación. 

In [119]:
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos')

cHora = 60*60/(tAutom + tVal)
print('Cantidad de Imagenes por Hora: ', math.ceil(cHora))

Tiempo de Automatizacion:  0.24 segundos
Cantidad de Imagenes por Hora:  5222


### Precision de Etiquetado:

In [120]:
eNoAnimal = pNoAnimal * (95.94/100) * (1.45/100)
print('Error de Imagenes sin Animales: ', round(eNoAnimal * 100,2), '%')

Error de Imagenes sin Animales:  0.98 %


In [121]:
eAnimal = pAnimal * (49.19/100) * ((1.41/100) + (41.79/100) * (5.22/100))
print('Error de Imagenes con Animales: ', round(eAnimal * 100,2), '%')

Error de Imagenes con Animales:  0.52 %


In [122]:
eAutomatico = eNoAnimal + eAnimal
print('Error Automatico: ', round(eAutomatico * 100,2), '%')

Error Automatico:  1.5 %


In [123]:
cErroresAutom = len(df) * eAutomatico
print('Cantidad de Errores: ', math.ceil(cErroresAutom))

Cantidad de Errores:  1783


In [124]:
eValidacion = pAnimal * ((16.7/100) + (3.4/100) + (10/100))/2 * pValAnimal + pNoAnimal * ((0.6/10) /2) * pValNoAnimal
print('Error de Validacion: ', round(eValidacion * 100,2), '%')

Error de Validacion:  3.79 %


In [125]:
cErroresVal = len(df) * pVal * eValidacion 
print('Cantidad de Errores: ', math.ceil(cErroresVal))

Cantidad de Errores:  1492


In [126]:
eErrores = eAutomatico + eValidacion
print('Error Total: ', round(eErrores * 100,2), '%')

Error Total:  5.29 %


In [127]:
cErrores = cErroresAutom + cErroresVal
print('Cantidad de Errores: ', math.ceil(cErrores))

Cantidad de Errores:  3274


### Costos del Proceso:

- Costo Falso Negativo: $300
- Costo Falso Positivo: $1000
- Costo Error en Especies: $1300
- Costo Error en Cantidad: $650

In [128]:
coErrorNoAnimal = eNoAnimal * 300
print('Costo de Error de Imagenes sin Animales: $', round(coErrorNoAnimal,2))

Costo de Error de Imagenes sin Animales: $ 2.95


In [129]:
coErrorAnimal = pAnimal * pGuanaco * ((1.41/100) * 1300 + (41.79/100) * (5.22/100) * 650)
print('Costo de Error de Imagenes con Animales: $', round(coErrorAnimal,2))

Costo de Error de Imagenes con Animales: $ 4.67


In [130]:
coErrorAutomatico = coErrorNoAnimal + coErrorAnimal
print('Costo de Error Automatico: $', round(coErrorAutomatico,2))

Costo de Error Automatico: $ 7.62


In [131]:
coErrorVal = pAnimal * pValAnimal * ((1.41/100) * 300 + (41.79/100) * (5.22/100) * 650)/2 + pNoAnimal * pValNoAnimal * ((0.6/100)/2) * 1000
print('Costo de Error de Validacion: $', round(coErrorVal,2))

Costo de Error de Validacion: $ 2.44


In [132]:
coOperacionalVal = pVal * tVal * 10730/(8*60*60)
print('Costo Operacional de Validacion: $', round(coOperacionalVal,2))

Costo Operacional de Validacion: $ 0.06


In [133]:
coServicio = 1100000
print('Costo de Servicio: $', round(coServicio,2))

Costo de Servicio: $ 1100000


In [134]:
coTotal = len(df) * (coErrorAutomatico + coErrorVal + coOperacionalVal) + coServicio
print('Costo Total: $', round(coTotal,2))

Costo Total: $ 2301359.31


In [135]:
coImagen = coTotal/len(df)
print('Costo por Imagen: $', round(coImagen,2))

Costo por Imagen: $ 19.38


## KPIs

### Tiempo Total del Proceso

In [136]:
print('Tiempo Total: ')
print('- Clasificacion Manual: 152.75 horas')
print('- Solucion Semi-Automatica:', round(TTotal,2), 'horas')
print('Variacion Porcentual: ', round((TTotal - 152.75)/152.75 * 100,2), '%')

Tiempo Total: 
- Clasificacion Manual: 152.75 horas
- Solucion Semi-Automatica: 22.74 horas
Variacion Porcentual:  -85.11 %


In [137]:
print('Jornadas Laborales:')
print('- Clasificacion Manual: 19.1 jornadas')
print('- Solucion Semi-Automatica:')
print('   - Jornadas Ocupadas: ', round(TVal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TVal/8 - 19.1)/19.1 * 100,2), '%')
print('    - Jornadas Totales: ', round(TTotal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TTotal/8 - 19.1)/19.1 * 100,2), '%')

Jornadas Laborales:
- Clasificacion Manual: 19.1 jornadas
- Solucion Semi-Automatica:
   - Jornadas Ocupadas:  1.9 jornadas
    Variacion Porcentual:  -90.28 %
    - Jornadas Totales:  2.8 jornadas
    Variacion Porcentual:  -85.11 %


### Cantidad de Imagenes Analizadas por Hora:

In [138]:
print('Cantidad de Imagenes por Hora: ')
print('- Clasificacion Manual: 777 imagenes/hr')
print('- Solucion Semi-Automatica: ', math.ceil(cHora), 'imagenes/hr')
print('Variacion Porcentual: ', round((cHora - 777)/777 * 100,2), '%')

Cantidad de Imagenes por Hora: 
- Clasificacion Manual: 777 imagenes/hr
- Solucion Semi-Automatica:  5222 imagenes/hr
Variacion Porcentual:  572.03 %


### Precision de Etiquetado:

In [139]:
print('Tasa de Error: ')
print('- Clasificacion Manual: 10%')
print('- Solucion Semi-Automatica: ', round(eErrores * 100,2), '%')
print('Variacion Porcentual: ', round((eErrores * 100 - 10)/10 * 100,2), '%')

Tasa de Error: 
- Clasificacion Manual: 10%
- Solucion Semi-Automatica:  5.29 %
Variacion Porcentual:  -47.06 %


In [140]:
print('Cantidad de Errores: ')
print('- Clasificacion Manual: 11877 errores')
print('- Solucion Semi-Automatica: ', math.ceil(cErrores), 'errores')
print('Variacion Porcentual: ', round((cErrores - 11877)/11877 * 100,2), '%')

Cantidad de Errores: 
- Clasificacion Manual: 11877 errores
- Solucion Semi-Automatica:  3274 errores
Variacion Porcentual:  -72.44 %


### Costo del Proceso:

In [141]:
print('Costo por Imagen:')
print('- Clasificacion Manual: $ 56.5')
print('- Solucion Semi-Automatica: $', round(coImagen,2))
print('Variacion Porcentual: ', round((coImagen - 56.5)/56.5* 100,2), '%')

Costo por Imagen:
- Clasificacion Manual: $ 56.5
- Solucion Semi-Automatica: $ 19.38
Variacion Porcentual:  -65.7 %


In [142]:
print('Costo del Proceso:')
print('- Clasificacion Manual: $ 6722325.4')
print('- Solucion Semi-Automatica: $', round(coTotal,2))
print('Variacion Porcentual: ', round((coTotal - 6722325.4)/6722325.4 * 100,2), '%')

Costo del Proceso:
- Clasificacion Manual: $ 6722325.4
- Solucion Semi-Automatica: $ 2301359.31
Variacion Porcentual:  -65.77 %


### Ratio de Imagenes Utiles:

In [143]:
print('Ratio de Imagenes Utiles: ')
print('- Clasificacion Manual: 0.79%')
print('- Solucion Semi-Automatica: 100%')
print('Variacion Porcentual: ', round((100 - 0.79)/0.79 * 100,2), '%')

Ratio de Imagenes Utiles: 
- Clasificacion Manual: 0.79%
- Solucion Semi-Automatica: 100%
Variacion Porcentual:  12558.23 %


# Resultados:

In [144]:
df = pd.read_csv('resultados.csv')

In [145]:
df["Solucion Semi-Automatica"]=[round(tTotal,2), round(TVal/8,1), round(TTotal/8,1), math.ceil(cHora), round(eErrores * 100,2), math.ceil(cErrores), round(coImagen,2), 100 ]

In [146]:
df["Variacion Porcentual (%)"] = round((df["Solucion Semi-Automatica"] - df["Clasificacion Manual"]) / df["Clasificacion Manual"] * 100, 2)
df

,KPI,Clasificacion Manual,Estimacion Automatica,Estimacion Semi-Automatica,Solucion Semi-Automatica,Variacion Porcentual (%)
0,Segundos por Imagen,4.63,0.56,1.84,0.69,-85.10
1,Jornadas Ocupadas,19.10,0.00,1.60,1.90,-90.05
2,Jornadas Totales,19.10,2.30,7.60,2.80,-85.34
3,Imagenes por Hora,777.00,6480.00,1962.00,5222.00,572.07
4,Tasa de Error,10.00,16.30,6.13,5.29,-47.10
5,Cantidad de Errores,11877.00,19357.00,7281.00,3274.00,-72.43
6,Costo por Imagen,56.50,19.45,22.69,19.38,-65.70
7,Ratio de Imagenes Utiles,0.79,100.00,100.00,100.00,12558.23
